<a href="https://colab.research.google.com/github/wksmirnowa/compling_homeworks/blob/master/HW06_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Векторные представления

## Импорты и настройка колаба для работы с fasttext с rusvectores

In [0]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

In [0]:
!conda update --all

In [1]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 6.1MB/s 
     |████████████████████████████████| 378kB 52.9MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=868528 sha256=bb6a786f08567a4b49650db9c5f2f8c6657309ab2f5c52e708391f5cd44c3500
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [2]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 1.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import pandas as pd
from lxml import html
import numpy as np
import gensim
from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from gensim.models import FastText
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from scipy import spatial

morph = MorphAnalyzer()
mystem = Mystem()

punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian')) | {'gt', }
added_stops = {'весь', 'это', 'наш', 'оно', 'итак', 'т.п', 'т.е', 'мало', 'меньше', 'ещё', 'слишком', 'также',
                   'ваш', 'б', 'хм', 'который', 'свой', 'не', 'мочь', 'однако', 'очень', 'п', 'благодаря', 'кроме'}
stops = stops.union(added_stops)

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Тексты

In [0]:
#Корпус гуманитарных текстов для обучения w2v модели

corpus_hum = open('/content/drive/My Drive/corpus_hum.txt').read().splitlines()

corpus_hum_norm = [normalize(text) for text in corpus_hum]

In [0]:
corpus_hum_norm = [text for text in corpus_hum_norm if text]

In [0]:
#Создание корпуса перефразирования 

corpus_xml = html.fromstring(open('/content/drive/My Drive/paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [0]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [0]:
data.head(2)

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...


# Задание 1.

## Векторизация

1) Векторизуйте тексты с помощью Word2vec модели, обученной самостоятельно. Word2Vec нужно обучить на отдельном корпусе (не на парафразах). Можно взять данные из семинара или любые другие. 

2) Векторизуйте тексты с помощью и с помощью модели, взятой с rusvectores (любой). Я взяла fasttext, обученный на Araneum.

### Подготовка

In [0]:
#соответствие тэгов майстема тэгам Udpipe 

mapping = {}

for line in open('/content/drive/My Drive/ru-rnc.map.txt'):
    ms, ud = line.strip('\n').split()
    mapping[ms] = ud

In [0]:
#проставление тегов нормализированным словам

def normalize_mystem(text):
    tokens = []
    norm_words = mystem.analyze(text)
    for norm_word in norm_words:
        if 'analysis' not in norm_word:
            continue
            
        if not len(norm_word['analysis']):
            lemma = norm_word['text']
            pos = 'UNKN'
        else:
            lemma = norm_word["analysis"][0]["lex"].lower().strip()
            pos = norm_word["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
        pos = mapping[pos]
        tokens.append(lemma+'_'+pos)

    return tokens

In [0]:
def get_embedding(text, model, dim):
    text = text.split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total)
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [0]:
def logreg(X_text, y=y, random_state=None, C=1000, max_iter=1000):
  train_X, valid_X, train_y, valid_y = train_test_split(X_text, y, random_state=random_state)
  clf_LR = LogisticRegression(C=C, max_iter=max_iter)
  return clf_LR

In [0]:
def cross_val(clf, X_text, y=y, scoring='f1_micro', cv=5):
  cross = cross_val_score(clf, X_text, y, scoring=scoring, cv=cv)
  print(f'Cross-validation: {cross}', f'F-score: {np.mean(cross)}', sep='\n')

### Word2vec

In [0]:
w2v = gensim.models.Word2Vec([text.split() for text in corpus_hum_norm], size=50, sg=1)

In [19]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [0]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [0]:
y = data.label.values

In [0]:
clf_w2v = logreg(X_text_w2v, random_state=42)

In [43]:
cross_val(clf_w2v, X_text_w2v) #Word2Vec LogisticRegression

Cross-validation: [0.42738589 0.46957123 0.4982699  0.43114187 0.42145329]
F-score: 0.44956443500026316


### Araneum Fasttext

In [31]:
!wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2020-04-25 12:18:19--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  11.1MB/s    in 4m 1s   

2020-04-25 12:22:21 (10.7 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]



In [0]:
import tarfile
fasttext_file = tarfile.open('/content/araneum_none_fasttextcbow_300_5_2018.tgz', 'r')
fasttext_file.extractall()

In [0]:
fasttext_araneum = gensim.models.KeyedVectors.load("araneum_none_fasttextcbow_300_5_2018.model")

In [0]:
dim = 50
X_text_1_fasttext_araneum = np.zeros((len(data['text_1_norm']), dim))
X_text_2_fasttext_araneum = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_fasttext_araneum[i] = get_embedding(text, fasttext_araneum, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_fasttext_araneum[i] = get_embedding(text, fasttext_araneum, dim)

In [0]:
X_text_fasttext_araneum = np.concatenate([X_text_1_fasttext_araneum, X_text_2_fasttext_araneum], axis=1)

In [0]:
clf_fa = logreg(X_text_fasttext_araneum, random_state=42)

In [49]:
cross_val(clf_fa, X_text_fasttext_araneum) #Fasttext Areneum LogisticRegression

Cross-validation: [0.40940526 0.40940526 0.40899654 0.40899654 0.40899654]
F-score: 0.4091600262267464


У обученного мною word2vec показатели чуть лучше, 
чем у предобученного на araneum фасттексте, но разница получилась совсем незначительная. Вероятно, это связано с тем, что предобученная на Araneum модель имела 5 млрд слов в обучении, поэтому она не сильно отстает от моей модели.

# Задание 2.

1)	Преобразуйте тексты в векторы в каждой паре с помощью SVD, SVD применяйте к данным напрямую.

In [0]:
cv = CountVectorizer(min_df=3, max_df=0.3, max_features=1000)
X = cv.fit_transform(pd.concat([data['text_1_norm'], data['text_2_norm']]))

In [0]:
X.shape

(14454, 1000)

In [87]:
svd = TruncatedSVD(25)
svd.fit(X)

TruncatedSVD(algorithm='randomized', n_components=25, n_iter=5,
             random_state=None, tol=0.0)

In [0]:
X_text_1_svd = svd.transform(cv.fit_transform(data['text_1_norm']))
X_text_2_svd = svd.transform(cv.fit_transform(data['text_2_norm']))

2)	Преобразуйте тексты в векторы в каждой паре с помощью  NMF, NMF применяйте к данным напрямую

In [89]:
nmf = NMF(25)
nmf.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=25, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
X_text_1_nmf = nmf.transform(cv.fit_transform(data['text_1_norm']))
X_text_2_nmf = nmf.transform(cv.fit_transform(data['text_2_norm']))

3)	Преобразуйте тексты в векторы в каждой паре с помощью Word2Vec (свой и русвекторовский)

4)	Преобразуйте тексты в векторы в каждой паре с помощью Fastext.

In [55]:
fasttext = gensim.models.FastText([text.split() for text in corpus_hum_norm], size=50, 
                                   min_n=4, max_n=8) 
dim = 50
X_text_1_fasttext = np.zeros((len(data['text_1_norm']), dim))
X_text_2_fasttext = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_fasttext[i] = get_embedding(text, fasttext, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_fasttext[i] = get_embedding(text, fasttext, dim)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


5)	У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). 

In [0]:
def make_cosines(X_text_1, X_text_2):
  cosines = []
  for pair1, pair2 in zip(X_text_1, X_text_2):
    if pair1.any() and pair2.any():
      cosines.append(spatial.distance.cosine(pair1, pair2))
    else:
      cosines.append(0)
  pairs = np.array(cosines)
  return pairs


In [0]:
data['svd'] = make_cosines(X_text_1_svd, X_text_2_svd)
data['nmf'] = make_cosines(X_text_1_nmf, X_text_2_nmf)
data['w2v'] = make_cosines(X_text_1_w2v, X_text_2_w2v)
data['fasttext_araneum'] = make_cosines(X_text_1_fasttext_araneum, X_text_2_fasttext_araneum)
data['fasttext'] = make_cosines(X_text_1_fasttext, X_text_2_fasttext)

In [0]:
vectors = data[['svd', 'nmf', 'w2v', 'fasttext_araneum', 'fasttext']]

In [0]:
clf_LR = LogisticRegression(C=1000, max_iter=1000)

In [93]:
cross_val(clf_LR, vectors) #Оценим все модели

Cross-validation: [0.54702628 0.59820194 0.6083045  0.46020761 0.49065744]
F-score: 0.5408795531881291


Максимальный результат f-метрики, которого мне удалось добиться путем изменения параметров векторизации – 0.5408795531881291